# Regulatory Reporting Dashboard
## Step 1 – Data Cleaning and Exploration
Goal: Understand the PaySim transaction dataset, clean the data, and generate initial insights to support regulatory reporting automation.


In [16]:
import pandas as pd

pd.set_option('display.max_columns', None)

# load data
df = pd.read_csv("../data/PaySim.csv")

# check first five rows
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [17]:
df.info()
df.describe()
df.isna().sum()
df.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


np.int64(0)

- Dataset contains ≈ 6,362,620 rows and 11 columns.  
- No missing values detected.  
- Column `isFraud` and `isFlaggedFraud` indicate fraud and flagged transactions.  
- `step` represents time in hours (1 step ≈ 1 hour in simulation).  

In [18]:
# delete duplicate rows
df = df.drop_duplicates()

# make sure amount is positive
df = df[df['amount'] > 0]

# reset index
df.reset_index(drop=True, inplace=True)

In [19]:
df.columns = [col.lower() for col in df.columns]
df.rename(columns={'nameorig': 'sender',
                   'namedest': 'receiver'}, inplace=True)
df.head()

,step,type,amount,sender,oldbalanceorg,newbalanceorig,receiver,oldbalancedest,newbalancedest,isfraud,isflaggedfraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [20]:
fraud_rate = df['isfraud'].mean()*100
flag_rate = df['isflaggedfraud'].mean()*100
print(f"Fraud Rate: {fraud_rate:.3f}%")
print(f"Flagged Fraud Rate: {flag_rate:.3f}%")

fraud_by_type = df.groupby('type')['isfraud'].mean().sort_values(ascending=False)
fraud_by_type

Fraud Rate: 0.129%
Flagged Fraud Rate: 0.000%


type
TRANSFER    0.007688
CASH_OUT    0.001832
CASH_IN     0.000000
DEBIT       0.000000
PAYMENT     0.000000
Name: isfraud, dtype: float64

- Overall fraud rate ≈ 0.13%
- TRANSFER transactions show highest fraud ratio.

In [21]:
import numpy as np

df['high_value_tx'] = np.where(df['amount'] > 200000, 1, 0)
df['risk_score'] = (df['isfraud']*5 + df['isflaggedfraud']*3 + df['high_value_tx'])
df[['amount','isfraud','isflaggedfraud','high_value_tx','risk_score']].head()

,amount,isfraud,isflaggedfraud,high_value_tx,risk_score
0,9839.64,0,0,0,0
1,1864.28,0,0,0,0
2,181.00,1,0,0,5
3,181.00,1,0,0,5
4,11668.14,0,0,0,0


In [22]:
output_path = "../data/cleaned_transactions.csv"
df.to_csv(output_path, index=False)